In [ ]:
# standard imports
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import RobustScaler


In [ ]:
# environment configuration
# check if running in colab

if "google.colab" in sys.modules:
  from google.colab import drive
  print("Running in Colab")
  drive.mount('/content/drive')

  # EDIT THIS: Your exact folder path in Drive
  PROJECT_ROOT = "/content/drive/MyDrive/Colab Notebooks/headway-prediction"
else:
    print("💻 Running in Local Environment")
    # Assuming notebook is in /notebooks, root is one level up
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# system setup path
if PROJECT_ROOT not in sys.path:
  sys.path.append(PROJECT_ROOT)
  print(f"added to sys.path: {PROJECT_ROOT}")

print(f"Project Root: {PROJECT_ROOT}")

Running in Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
added to sys.path: /content/drive/MyDrive/Colab Notebooks/headway-prediction
Project Root: /content/drive/MyDrive/Colab Notebooks/headway-prediction


In [ ]:
# Validate Imports
try:
    from src.config import Config
    from src.data.dataset import SubwayDataGenerator
    from src.models.st_convnet_v2 import HeadwayConvLSTM
    from src.evaluator import Evaluator
    print("✅ Success: All custom 'src' modules imported.")
except ImportError as e:
    print(f"❌ IMPORT ERROR: {e}")

✅ Success: All custom 'src' modules imported.


# Data Loading

In [ ]:
# data loading and scaling
config = Config()
config.DATA_DIR = os.path.join(PROJECT_ROOT, "data")
print(f"Loading data from {config.DATA_DIR}")

# 1 instantiate data generator
data_gen = SubwayDataGenerator(config)

# 2 load raw .npy files (no normalization - we'll use RobustScaler)
data_gen.load_data(normalize=False)
print(f"Raw max headway values: {data_gen.headway_data.max():.2f} min")

# 3 fit robust scaler (training split only)
# we fit on the first 60% to avoid leakage
total_timesteps = len(data_gen.headway_data)
train_limit = int(total_timesteps * 0.6)

print(f"\nFitting RobustScaler on first {train_limit} steps")
scaler = RobustScaler()
flat_train = data_gen.headway_data[:train_limit].reshape(-1, 1)
scaler.fit(flat_train)

# 4. Transform All Data
print("Transforming Headway and Schedule Data")
data_gen.headway_data = scaler.transform(data_gen.headway_data.reshape(-1, 1)).reshape(data_gen.headway_data.shape)
# use same scaler for schedule to match units
data_gen.schedule_data = scaler.transform(data_gen.schedule_data.reshape(-1, 1)).reshape(data_gen.schedule_data.shape)

# 5 save scaler for inference
scaler_path = os.path.join(PROJECT_ROOT, "models", "robust_scaler.pkl")
os.makedirs(os.path.dirname(scaler_path), exist_ok=True)
joblib.dump(scaler, scaler_path)  # ← FIXED: Actually save the scaler
print(f"✅ Scaler saved to {scaler_path}")

print(f"\nScaled data range: [{data_gen.headway_data.min():.2f}, {data_gen.headway_data.max():.2f}]")

Loading data from /content/drive/MyDrive/Colab Notebooks/headway-prediction/data
Loading data from /content/drive/MyDrive/Colab Notebooks/headway-prediction/data...
Headway Shape: (264222, 66, 2, 1)
Schedule Shape: (264222, 2, 1)
Raw max headway values: 30.0 min (should be ~30.0)

Fitting RobustScaler on first 158533 steps
Transforming Headway and Schedule Data
Scaler saved to /content/drive/MyDrive/Colab Notebooks/headway-prediction/models/robust_scaler.pkl


# Baseline Experiment Configuration

In [ ]:
# 3 baseline config
# override config for the baseline experiment
config.LOOKBACK_MINS = 30
config.FORECAST_MINS = 15
config.BATCH_SIZE = 64
config.EPOCHS = 20
config.FILTERS = 64

print(f'--- Baseline Run Config --- ')
print(f'lookback: {config.LOOKBACK_MINS}')
print(f'batch: {config.BATCH_SIZE}')
print(f'epochs: {config.EPOCHS}')
print(f'filters: {config.FILTERS}')

# create tf datasets
# temporal split 60^ train, 20% val and test
train_end = int(0.6 * total_timesteps)
val_end = int(0.8 * total_timesteps)

print(f"\ncreating datasets...")
train_ds = data_gen.make_dataset(start_index=0, end_index=train_end, shuffle=True)
val_ds = data_gen.make_dataset(start_index=train_end, end_index=val_end, shuffle=False)
test_ds = data_gen.make_dataset(start_index=val_end, end_index=None, shuffle=False)

# quick shape check
for inputs, targets in train_ds.take(1):
  print(f"Input headway shape: {inputs['headway_input'].shape}")
  print(f"Target shape: {targets.shape}")

--- Baseline Run Config --- 
lookback: 30
batch: 64
epochs: 20
filters: 64

creating datasets...
Creating dataset from index 0 to 158533
Creating dataset from index 158533 to 211377
Creating dataset from index 211377 to 264177
Input headway shape: (64, 30, 66, 2, 1)
Target shape: (64, 15, 66, 2, 1)


# Model Build and Training

In [ ]:
# train baseline
print(f"\nBuilding V2 Architecture")
builder = HeadwayConvLSTM(
    n_stations=config.NUM_STATIONS,
    lookback=config.LOOKBACK_MINS,
    forecast=config.FORECAST_MINS,
    output_activation='linear'  # ← Use linear for RobustScaler (unbounded range)
)

model = builder.build_model()
model.summary()

# compile (huber loss - robust to outliers)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0)
loss_fn = tf.keras.losses.Huber(delta=1.0)
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['mae', 'mse'],
    jit_compile=False  # Disable XLA for compatibility
)

# callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
]

print("\n🚀 Starting Training...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config.EPOCHS,
    callbacks=callbacks
)

# save model (FIXED: added missing comma in path join)
model_path = os.path.join(PROJECT_ROOT, "models", "v2_baseline_30min.keras")
os.makedirs(os.path.dirname(model_path), exist_ok=True)
model.save(model_path)
print(f"✅ Model saved to {model_path}")


Building V2 Architecture
Starting Training...
Epoch 1/20
2477/2477 ━━━━━━━━━━━━━━━━━━━━ 499s 197ms/step - loss: 0.1695 - mae: 0.4089 - mse: 0.3959 - val_loss: 0.1649 - val_mae: 0.3905 - val_mse: 0.3955 - learning_rate: 0.0010
Epoch 2/20
 145/2477 ━━━━━━━━━━━━━━━━━━━━ 6:46 175ms/step - loss: 0.1515 - mae: 0.3731 - mse: 0.3572

# Model Evaluation

In [ ]:
# evaluate
print(f"Evaluating Baseline")
evaluator = Evaluator(config)

# 1. plot loss curves (scaled units)
evaluator.plot_loss(history)

# 2. plot predcition (real minutes)
print(f"Visualizing Sample Prediction")
evaluator.plot_spatiotemporal_prediction(
    model,
    test_ds,
    scaler=scaler,
    sample_idx=0,
    direction=0
)

In [ ]:
# Smoke Test: Verify model trains quickly with synthetic data
import time
from src.models.st_convnet_v2 import HeadwayConvLSTM

# 1. Setup parameters matching your data
BATCH_SIZE = 32
N_STATIONS = config.NUM_STATIONS  # Use actual station count
LOOKBACK = 30
FORECAST = 15

# 2. Generate Random Data in RAM (No Disk I/O)
print("Generating synthetic data in RAM...")
X_headway = np.random.randn(BATCH_SIZE * 4, LOOKBACK, N_STATIONS, 2, 1).astype('float32')
X_schedule = np.random.randn(BATCH_SIZE * 4, FORECAST, 2, 1).astype('float32')
Y_target = np.random.randn(BATCH_SIZE * 4, FORECAST, N_STATIONS, 2, 1).astype('float32')

# 3. Build & Compile
print("Building model...")
builder = HeadwayConvLSTM(
    n_stations=N_STATIONS, 
    lookback=LOOKBACK, 
    forecast=FORECAST,
    output_activation='linear'  # Match training setup
)
test_model = builder.build_model()
test_model.compile(optimizer='adam', loss='mse', jit_compile=False)

# 4. Time the Training Loop
print("⏱️ Starting Smoke Test (4 batches, 1 epoch)...")
start = time.time()
test_model.fit([X_headway, X_schedule], Y_target, epochs=1, batch_size=BATCH_SIZE, verbose=1)
end = time.time()

print(f"\n✅ Total Time: {end - start:.2f} seconds")
print(f"   Expected: ~10-30s on GPU, ~60-120s on CPU")
print(f"   If > 5 min: CuDNN is NOT being used (check activations)")

In [ ]:
# Create a tiny dataset (just for a speed test)
print("Creating MINI dataset for speed test...")
train_ds = data_gen.make_dataset(start_index=0, end_index=2000, shuffle=True)
val_ds   = data_gen.make_dataset(start_index=2000, end_index=2500, shuffle=False)

# Run training
model.fit(train_ds, validation_data=val_ds, epochs=5)